In [ ]:
# Create a JIRA client
from os import environ
from atlassian import Jira

jira_api_token = environ.get("JIRA_TOKEN", "")
jira_url = environ.get("JIRA_URL", "")
client = Jira(url=jira_url, token=jira_api_token)

In [ ]:
# Import our local modules
from jiraissues import Issue, issue_cache
from summarizer import summarize_issue, get_chat_model, rollup_contributors

In [ ]:
epic_to_summarize = "OCTO-2"

In [ ]:
# Get the existing summaries from the Jira issues
child_inputs = []
epic = issue_cache.get_issue(client, epic_to_summarize)
for child in epic.children:
    issue = issue_cache.get_issue(client, child.key)
    text = f"{issue}\n"
    text += summarize_issue(issue, max_depth=1)
    child_inputs.append({"issue": issue, "summary": text})

# Sort the issues by key
child_inputs.sort(key=lambda x: x["issue"].key)

In [ ]:
# Generate the individual exec summaries
import textwrap
llm = get_chat_model("meta-llama/llama-3-70b-instruct", max_new_tokens=2048)
for item in child_inputs:
    data = f"""\
{item["issue"]}
{item["summary"]}
Contributors: {', '.join(c.display_name for c in item["issue"].contributors)}"""
    prompt = f"""\
Condense the following technical status update into a short, high-level summary for an engineering leader.
Focus on the high-level objective, keeping the technical detail to a minimum.
Where possible, avoid mentioning specific issue IDs.

{data}

Please provide your converted summary with no formatting or bullet points:
"""
    summary = llm.invoke(prompt, stop=["<|endoftext|>"])
    item["exec_summary"] = textwrap.fill(summary).strip()

In [ ]:
for item in child_inputs:
    issue = item["issue"]
    print(f"**{issue.key} - {issue.summary}**")
    print(item["exec_summary"])
    contributors = sorted(rollup_contributors(item["issue"]), key=lambda x: x.display_name.split()[-1])
    if contributors:
        print(f"Contributors: {', '.join([c.display_name for c in contributors])}")
    print()

In [ ]:
# Generate the overall exec summary
prompt = f"""\
Given the following high-level summaries of our group's work, please provide a short, one-paragraph summary of this initiative for a corporate leader:

{"\n".join([item["exec_summary"] for item in child_inputs])}

Please provide just the summary paragraph, with no header.
"""
paragraph = llm.invoke(prompt, stop=["<|endoftext|>"])
print(paragraph.strip())
